This notebook aims to build a graph purely with KNN.

In [2]:
import numpy as np
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/mini_nusc', verbose=True)

# nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/nuscenes2', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.164 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
import torch_geometric as tgeo

tgeo.get_home_dir()

'/home/maximilian/.cache/pyg'

In [4]:
#Get set of scenes
scenes = nusc.scene
#Get first scenes
scene_0 = scenes[0]
# Get token of first frame
first_sample_token = scene_0['first_sample_token']

NUMBER_OF_SKIPPED_FRAMES = 10
for i in range(NUMBER_OF_SKIPPED_FRAMES):
    temp_sample = nusc.get('sample', first_sample_token)
    temp_token = temp_sample['next']
    first_sample_token = temp_token

sample_0 = nusc.get('sample', first_sample_token)
# Get tokens for 2 following frames
second_sample_token = sample_0['next']
sample_1 = nusc.get('sample', second_sample_token)
third_sample_token = sample_1['next']
sample_2 = nusc.get('sample', third_sample_token)

# Get LIDAR pointcloud
sensor = 'LIDAR_TOP'
lidar_top_data_0 = nusc.get('sample_data', sample_0['data'][sensor])
# Get LIDAR KF pointcloud
lidar_top_data_1 = nusc.get('sample_data', sample_1['data'][sensor])
# Get LIDAR KF pointcloud
lidar_top_data_2 = nusc.get('sample_data', sample_2['data'][sensor])

#Filter out car/vehicle object
pcl0_path, boxes0, _= nusc.get_sample_data(lidar_top_data_0['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
pcl1_path, boxes1, _= nusc.get_sample_data(lidar_top_data_1['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
pcl2_path, boxes2, _= nusc.get_sample_data(lidar_top_data_2['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)


In [5]:
# Get point clouds
import os.path as osp
from nuscenes.utils.data_classes import LidarPointCloud, Box

#Load Pointclouds
pc0 = LidarPointCloud.from_file(pcl0_path)
pc1 = LidarPointCloud.from_file(pcl1_path)
pc2 = LidarPointCloud.from_file(pcl2_path)

from utility import get_box_centers, filter_boxes

car_boxes_0 = filter_boxes(nusc, boxes = boxes0, categoryQuery= 'vehicle.car')
car_boxes_1 = filter_boxes(nusc, boxes = boxes1, categoryQuery= 'vehicle.car')
car_boxes_2 = filter_boxes(nusc, boxes = boxes2, categoryQuery= 'vehicle.car')

centers0 = get_box_centers(car_boxes_0)
centers1 = get_box_centers(car_boxes_1)
centers2 = get_box_centers(car_boxes_2)

# Special Shift parameter in meter
SPATIAL_SHIFT_TIMEFRAMES = 20
# Boxes 0 can stay at the current frame
# centers0
# Boxes 1 must be translated up by l meters
centers1 += np.array([0,0,SPATIAL_SHIFT_TIMEFRAMES])

# Boxes 2 must be translated up by 2*l meters
centers2 += np.array([0,0,2*SPATIAL_SHIFT_TIMEFRAMES])

# Add all centroids into one array
centers = np.empty((0,3))
centers = np.append(centers, centers0, axis=0)
centers = np.append(centers, centers1, axis=0)
centers = np.append(centers, centers2, axis=0)

In [6]:
# Build KNN Graph
from sklearn.neighbors import NearestNeighbors
from graph.graph_generation import Graph

# Add temporal edges / Make connections#####################
temporal_pointpairs = []

TEMPORAL_KNN_PARAM = 3
''' 
Defines the number of searched neighbors
'''

# connect frame-0-nodes with frame-1-nodes
for i in range(len(centers0)):
    center = centers0[i]
    center = np.expand_dims(center,axis=0)
    temp = np.append(centers1,center,axis=0)
    #Find nearest_neigbor
    nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
    temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)
    #Add indices into a list
    for index in temporal_indices[-1]:
        #adapt the index to the global indexing
        # temporal_pointpairs.append([i, index + len(centers0)])

        # find global indices and append them
        reference_node_global_index = np.argwhere(centers == center)[0,0]
        neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
        temporal_pointpairs.append([reference_node_global_index ,\
             neighbor_node_global_index ])

# connect frame-0-nodes with frame-2-nodes
for i in range(len(centers0)):
    center = centers0[i]
    center = np.expand_dims(center,axis=0)
    temp = np.append(centers2,center,axis=0)
    #Find nearest_neigbor
    nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
    temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)
    #Add indices into a list (The last entry belongs to center!)
    for index in temporal_indices[-1]:
        #adapt the index to the global indexing
        # temporal_pointpairs.append([i, index + len(centers0)])

        # find global indices and append them
        reference_node_global_index = np.argwhere(centers == center)[0,0]
        neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
        temporal_pointpairs.append([reference_node_global_index ,\
             neighbor_node_global_index ])

# connect frame-1-nodes with frame-2-nodes
for i in range(len(centers1)):
    center = centers1[i]
    center = np.expand_dims(center,axis=0)
    temp = np.append(centers2,center,axis=0)
    nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
    temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)

    # Test if the last input is the appended center point
    # assert (temporal_distances[-1] == temporal_distances[np.argwhere(temp == center)[0,0]]).all()

    for index in temporal_indices[-1]:
        #adapt the index to the global indexing
        # temporal_pointpairs.append([i + len(centers0) , index + len(centers0) + len(centers1) ])
        
        # find global indices and append them
        reference_node_global_index = np.argwhere(centers == center)[0,0]
        neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
        temporal_pointpairs.append([reference_node_global_index ,\
             neighbor_node_global_index ])


In [7]:
#Build graph in a more organised manner
from graph.graph_generation import SpatioTemporalGraph, Timeframe

SPATIAL_NEIGHBOR_NUMS = 5

nbrs_0 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers0)
# Frame t0
#Compute K nearest neighbors
spatial_distances_0, spatial_indices_0 = nbrs_0.kneighbors(centers0)
# Make a list of tuple pairs
spatial_pairs = [] 
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_0):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t0,curr_node_idx) , ((Timeframe.t0,neigbor_index)) ) )

#Frame t1
nbrs_1 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers1)
spatial_distances_1, spatial_indices_1 = nbrs_1.kneighbors(centers1)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_1):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t1,curr_node_idx) , ((Timeframe.t1,neigbor_index)) ) )

#Frame t2
nbrs_2 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers2)
spatial_distances_2, spatial_indices_2 = nbrs_2.kneighbors(centers2)
# Make a list of tuple pairs
for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_2):
    for neigbor_index in neigborhood_indices:
        spatial_pairs.append( \
            ( (Timeframe.t2,curr_node_idx) , ((Timeframe.t2,neigbor_index)) ) )

testgraph = SpatioTemporalGraph(spatial_pairs)

# Get all spatial Edges
spatial_pointpairs0 = []
for reference_node in testgraph._graph:
    if(reference_node[0]== Timeframe.t0):
        for neighbor_node in testgraph._graph[reference_node]:
            # print(neighbor_index[0])
            timestep, idx = neighbor_node[0],neighbor_node[1]
            if timestep == Timeframe.t0:
                spatial_pointpairs0.append([reference_node[1],idx])

print(np.shape(spatial_pointpairs0))
testarray = testgraph.get_spatial_pointpairs(Timeframe.t0)
assert spatial_pointpairs0 == testarray

spatial_pointpairs1 = testgraph.get_spatial_pointpairs(Timeframe.t1)
spatial_pointpairs2 = testgraph.get_spatial_pointpairs(Timeframe.t2)

(63, 2)


In [8]:
# Generate  Flow Labels for TrackingGNN
from groundtruth_generation.nuscenes_create_gt import generate_flow_labels
from utils.nuscenes_helper_functions import is_valid_box

car_boxes = [] 
for list in [car_boxes_0, car_boxes_1, car_boxes_2]:
    for box in list:
        car_boxes.append(box)

if len(car_boxes)== len(centers):
    print('length', len(car_boxes))
    # for box, center in zip(car_boxes, centers):
    for i in range(len(car_boxes)):
        box, center = car_boxes[i], centers[i]
        if not is_valid_box(box, center, num_frames = 3, spatial_shift_timeframes=SPATIAL_SHIFT_TIMEFRAMES):
            print('invalid boxes at ',i)
            print('box',box.center)
            print('center',center)

flow_labels = generate_flow_labels(nusc,temporal_pointpairs, car_boxes, centers)


length 33


In [11]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)


In [ ]:

data_list = []
spatio_temporal_edge_indices = temporal_pointpairs \
        + spatial_pointpairs0
data_list.append(Data(x= centers,edge_index= spatio_temporal_edge_indices, y= flow_labels))
loader = DataLoader(data_list, batch_size=32)